# Reportagem: Execução orçamentária com Cultura pela Prefeitura de Fortaleza entre 2010 a 2022

Este repositório reúne códigos construídos para coleta e análise de dados para a realização da reportagem "[Cultura na gestão Sarto é marcada por recordes negativos de investimentos na área](https://mais.opovo.com.br/reportagens-especiais/2023/04/10/cultura-na-gestao-sarto-e-marcada-por-recordes-negativos-de-investimentos-na-area.html)", publicada no [O POVO+](https://mais.opovo.com.br/) em 10 de abril de 2023 e produzida em parceria com o Vida & Arte, núcleo de Cultura do O POVO.

**ÍNDICE**

1. [Coleta, limpeza e checagem dos dados](#coleta-limpeza-checagem)

    1.1 [Despesas por função](#despesas-funcao)

    1.2 [Despesas por área de atuação](#despesas-area-atuacao)


2. [Análise e visualizações](#analise-e-visualizacoes)

    2.1 [Valor empenhado para a Cultura por ano em Fortaleza-CE, entre 2010 e 2022 e Percentual do valor do orçamento empenhado para o setor cultural em relação às demais funções de governo entre 2010 e 2022](#valor-empenhado-demais-funcoes)

    2.2 [Valor empenhado em Cultura em cada ano (2010-2022) em relação ao que estava previsto na Lei Orçamentária Anual (LOA)](#valor-empenhado-loa)

    2.3 [Valores Empenhados, Liquidados e Pagos para a Cultura, entre 2014 e 2022](#valores-empenhados-liquidados-pagos)

## 1. Coleta, limpeza e checagem dos dados <a name="coleta-limpeza-checagem"></a>

In [22]:
import os
import pandas as pd
import numpy as np

### 1.1 Despesas por função <a name="despesas-funcao"></a>

O código utilizado para coleta, via *Selenium*, está disponível abaixo como comentário. Neste arquivo apenas realizamos a leitura e tratamento dos dados previamente coletados. 

In [23]:
arquivos = 'arquivos/'


Código utilizado para coleta (para saber mais sobre as configurações da biblioteca utilizada, consulte [este link](https://pypi.org/project/selenium/): 


``` python 
import pandas as pd

from selenium.webdriver import Chrome, ChromeOptions, Firefox
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import time 
from selenium.webdriver.firefox.options import Options


for a in range(2010,2023):
    print(a)
    meses = ['dez', 'nov', 'out', 'set', 'ago', 'jul', 'jun', 'mai', 'abr','mar', 'fev', 'jan']
    colunas = ['descrição da despesa','Valor Anual Previsto na LOA','valor mensal empenhado','Valor Acumulado']
    df_ano_mes = []
    for i,m in enumerate(meses):
        print(m)
        url = f'https://transparencia.fortaleza.ce.gov.br/index.php/despesa/exibirDespesasPorFuncao/{a}#tabs-{m}'

        options = Options()
        options.headless = True
        webdriver = Firefox(executable_path='/home/alexandre/bin/geckodriver', options=options)

        webdriver.get(url)
        time.sleep(3)

        table = webdriver.find_elements(By.CLASS_NAME, 'table')[11-i]
        
        rows = table.find_elements(By.TAG_NAME, "tr") 
        cr = 0
        lista_dados = []
        for row in rows[1:]:
            col = row.find_elements(By.TAG_NAME, "td")
            lista_dados.append({
                                'ano':a,
                                'mes':m,
                                colunas[0]:col[0].text,
                                colunas[1]:col[1].text,
                                colunas[2]:col[2].text,
                                colunas[3]:col[3].text,
                                })
        df = pd.DataFrame(lista_dados)

        df_ano_mes.append(df)

        df = pd.concat(df_ano_mes)
        df.to_csv(arquivos + f'df_{a}.csv')
        webdriver.close()
        
```




In [24]:
dict_meses = {'dez':12,
              'nov':11, 
              'out':10,
              'set':9, 
              'ago':8, 
              'jul':7, 
              'jun':6,
              'mai':5,
              'abr':4,
              'mar':3,
              'fev':2,
              'jan':1
             }

In [25]:
pd.options.display.float_format = '{:.2f}'.format

In [26]:
!pip install calculadora_do_cidadao

You should consider upgrading via the '/home/alexandre/anaconda3/envs/credos/bin/python -m pip install --upgrade pip' command.


In [27]:
from calculadora_do_cidadao import Ipca
from datetime import date

In [28]:
ipca = Ipca()

In [29]:
path_scrap = 'web_scrap_selenium'

In [9]:
lista_df = []

for f in os.listdir(path_scrap):
    lista_df.append(pd.read_csv(path_scrap+'/'+f))

wsDf = pd.concat(lista_df)[['ano', 'mes', 'descrição da despesa',
       'Valor Anual Previsto na LOA', 'valor mensal empenhado',
       'Valor Acumulado']]

FileNotFoundError: [Errno 2] No such file or directory: 'web_scrap_selenium'

In [13]:
wsDf['ano/mes'] = wsDf.apply(lambda x: f'{x.mes}/{x.ano}', axis=1)

In [14]:
wsDf['Valor Anual Previsto na LOA'] = wsDf['Valor Anual Previsto na LOA'].str.replace('.', '').str.replace(',', '.').astype(float)
wsDf['valor mensal empenhado'] = wsDf['valor mensal empenhado'].str.replace('.', '').str.replace(',', '.').astype(float)
wsDf['Valor Acumulado'] = wsDf['Valor Acumulado'].str.replace('.', '').str.replace(',', '.').astype(float)

C:\Users\55859\AppData\Local\Temp/ipykernel_2552/3375274452.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  wsDf['Valor Anual Previsto na LOA'] = wsDf['Valor Anual Previsto na LOA'].str.replace('.', '').str.replace(',', '.').astype(float)
C:\Users\55859\AppData\Local\Temp/ipykernel_2552/3375274452.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  wsDf['valor mensal empenhado'] = wsDf['valor mensal empenhado'].str.replace('.', '').str.replace(',', '.').astype(float)
C:\Users\55859\AppData\Local\Temp/ipykernel_2552/3375274452.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expression

In [15]:
wsDf['dtg'] = wsDf.apply(lambda x: pd.to_datetime(f'{x.ano}-{dict_meses[x.mes]}'), axis=1)

In [16]:
wsDf['valor mensal empenhado corrigido'] = wsDf.apply(lambda x: x['valor mensal empenhado'] * float(ipca.adjust(x['dtg'])), axis=1)
wsDf['valor Acumulado corrigido'] = wsDf.apply(lambda x: x['Valor Acumulado'] * float(ipca.adjust(x['dtg'])), axis=1)

NameError: name 'ipca' is not defined

In [17]:
wsDf.loc[wsDf['descrição da despesa'] != 'TOTAIS'][['ano','mes','valor mensal empenhado','valor mensal empenhado corrigido','dtg']]

KeyError: "['valor mensal empenhado corrigido'] not in index"

### 1.2 Despesas por área de atuação <a name="despesas-area-atuacao"></a>

A coleta dos dados foi feita com o webscraper, extensão para browsers. Para saber mais sobre a ferramenta, acesse [este link](https://www.webscraper.io/).

In [11]:
df = pd.read_csv('arquivos/despesas_por_area_de_atuacao.csv')

In [12]:
funcoes = ['ADMINISTRACAO', 'ASSISTENCIA SOCIAL', 'CIENCIA E TECNOLOGIA', 'COMERCIO E SERVICOS', 'COMUNICACOES',  'CULTURA', 'DESPORTO E LAZER', 'DIREITOS DA CIDADANIA', 'EDUCACAO', 'ENCARGOS ESPECIAIS', 'ENERGIA', 'ESSENCIAL A JUSTICA', 'GESTAO AMBIENTAL', 'HABITACAO', 'JUDICIARIA', 'LEGISLATIVA', 'PREVIDENCIA SOCIAL', 'RESERVA DE CONTINGENCIA', 'SANEAMENTO', 'SAUDE', 'SEGURANCA PUBLICA', 'TRABALHO', 'TRANSPORTE', 'URBANISMO']

In [13]:
df['tipo'] = np.where(df['função / subfunção'].isin(funcoes), 'função', 'subfunção')

In [14]:
df['ano'] = df['web-scraper-start-url'].apply(lambda x: x.split("exercicio=")[1].split("&")[0]) 

In [15]:
df['Dotação Orçamentária'] = df['Dotação Orçamentária'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Valor Empenhado'] = df['Valor Empenhado'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Valor Liquidado'] = df['Valor Liquidado'].str.replace('.', '').str.replace(',', '.').astype(float)
df['Valor Pago'] = df['Valor Pago'].str.replace('.', '').str.replace(',', '.').astype(float)

/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  
/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imp

In [16]:
df['dtg'] = df.ano.apply(lambda x: pd.to_datetime(f'{x}-12-31')) 

In [17]:
df['valor_pago_corrigido'] = df.apply(lambda x: x['Valor Pago'] * float(ipca.adjust(x['dtg'])), axis=1)
df['valor_empenhado_corrigido'] = df.apply(lambda x: x['Valor Empenhado'] * float(ipca.adjust(x['dtg'])), axis=1)
df['valor_liquidado_corrigido'] = df.apply(lambda x: x['Valor Liquidado'] * float(ipca.adjust(x['dtg'])), axis=1)

In [18]:
def getSubfunc(idx_ano):
    cont = idx_ano+1
    while df.iloc[cont].tipo == 'subfunção':
        cont+=1
        return df.iloc[idx_ano+1:cont].copy()

In [19]:
lista_subfuncoes = []
for y in list(df[df['função / subfunção'] == 'CULTURA'].index):
    lista_subfuncoes.append(getSubfunc(y))

In [20]:
subfuncoes_cultura = pd.concat(lista_subfuncoes)

In [21]:
subfuncoes_cultura

,web-scraper-order,web-scraper-start-url,função / subfunção,Dotação Orçamentária,Valor Empenhado,%(B/A)%,Valor Liquidado,Valor Pago,tipo,ano,dtg,valor_pago_corrigido,valor_empenhado_corrigido,valor_liquidado_corrigido
71,1676496995-491,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,41801420.00,26512144.03,"63,42",20563025.45,19885172.83,subfunção,2022,2022-12-31,20301606.91,27067359.74,20993655.08
207,1676496998-627,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,23992607.00,18593628.52,"77,50",16668781.43,16373598.47,subfunção,2021,2021-12-31,17683558.05,20081200.22,18002356.94
347,1676497001-767,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,37927557.00,34162964.22,"90,07",33041567.19,32994905.57,subfunção,2020,2020-12-31,39219837.73,40608266.34,39275302.69
487,1676497004-907,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,43945423.00,40134986.66,"91,33",38109576.55,37869534.93,subfunção,2019,2019-12-31,47047625.38,49862133.79,47345843.68
630,1676497007-1050,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,17328743.00,15876914.38,"91,62",15729878.00,15689393.70,subfunção,2018,2018-12-31,20331237.80,20574238.11,20383699.74
767,1676497010-1187,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,29008519.00,27659035.17,"95,35",27437682.64,27380270.86,subfunção,2017,2017-12-31,36809931.31,37184700.98,36887115.49
890,1676497013-1310,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,36537654.00,27374169.02,"74,92",27008773.64,26971005.10,subfunção,2016,2016-12-31,37328442.59,37886430.00,37380715.05
1013,1676497016-1433,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,24532065.00,21236437.43,"86,57",21132017.44,20724053.97,subfunção,2015,2015-12-31,30486083.95,31239824.75,31086217.91
1145,1676497019-1565,https://transparencia.fortaleza.ce.gov.br/inde...,ADMINISTRACAO GERAL,30293643.00,17959184.64,"59,28",17959184.64,15888448.95,subfunção,2014,2014-12-31,25867247.73,29238516.58,29238516.58


## 2. Análise e visualizações <a name="analise-e-visualizacoes"></a>

### 2.1 Valor empenhado para a Cultura por ano em Fortaleza-CE, entre 2010 e 2022 e Percentual do valor do orçamento empenhado para o setor cultural em relação às demais funções de governo entre 2010 e 2022 <a name="valor-empenhado-demais-funcoes"></a>


In [29]:
resumo_cultura_total = wsDf.loc[(wsDf['descrição da despesa'] == 'TOTAIS') | (wsDf['descrição da despesa'] == 'CULTURA')].pivot(index=['ano','mes'], columns ='descrição da despesa', values = 'valor mensal empenhado corrigido').reset_index()

KeyError: 'valor mensal empenhado corrigido'

In [30]:
resumo_cultura_total['pct'] = round(resumo_cultura_total['CULTURA']/resumo_cultura_total['TOTAIS']*100,2)

NameError: name 'resumo_cultura_total' is not defined

In [31]:
resumo_cultura_total.columns = ['Ano','Mẽs', 'Cultura', 'Todas as funções', 'Pct'] 

NameError: name 'resumo_cultura_total' is not defined

In [32]:
resumo_cultura_total

NameError: name 'resumo_cultura_total' is not defined

In [33]:
resumo_cultura_total_ano =  wsDf.loc[((wsDf['descrição da despesa'] == 'TOTAIS') | (wsDf['descrição da despesa'] == 'CULTURA')) & (wsDf['mes'] == 'dez')].pivot(index='ano', columns ='descrição da despesa', values = 'valor Acumulado corrigido').reset_index()

KeyError: 'valor Acumulado corrigido'

In [34]:
resumo_cultura_total_ano['pct'] = round(resumo_cultura_total_ano['CULTURA']/resumo_cultura_total_ano['TOTAIS']*100,2)

NameError: name 'resumo_cultura_total_ano' is not defined

In [35]:
resumo_cultura_total_ano.columns = ['Ano', 'Cultura', 'Todas as funções', 'Pct']

NameError: name 'resumo_cultura_total_ano' is not defined

In [36]:
resumo_cultura_total_ano

NameError: name 'resumo_cultura_total_ano' is not defined

In [37]:
resumo_cultura_total_ano_sem_correcao =  wsDf.loc[((wsDf['descrição da despesa'] == 'TOTAIS') | (wsDf['descrição da despesa'] == 'CULTURA')) & (wsDf['mes'] == 'dez')].pivot(index='ano', columns ='descrição da despesa', values = 'Valor Acumulado').reset_index()

In [38]:
resumo_cultura_total_ano_sem_correcao['pct'] = round(resumo_cultura_total_ano_sem_correcao['CULTURA']/resumo_cultura_total_ano_sem_correcao['TOTAIS']*100,2)
resumo_cultura_total_ano_sem_correcao.columns = ['Ano', 'Cultura _ Sem Correção', 'Todas as funções _ Sem correção', 'Pct  _ Sem correção']

In [39]:
resumo_cultura_total_ano_sem_correcao

,Ano,Cultura _ Sem Correção,Todas as funções _ Sem correção,Pct _ Sem correção
0,2010,31217869.22,3365966292.85,0.93
1,2011,41761489.58,3888179800.35,1.07
2,2012,50230328.42,4404254807.14,1.14
3,2013,36721858.05,4812040669.91,0.76
4,2014,44820159.52,5414958046.03,0.83
5,2015,32255792.19,5847827663.35,0.55
6,2016,46434616.84,6045848856.99,0.77
7,2017,43347750.02,6525657824.35,0.66
8,2018,60060690.19,6876456829.67,0.87
9,2019,72309943.34,8207466751.46,0.88


In [40]:
pd.merge(resumo_cultura_total_ano_sem_correcao, resumo_cultura_total_ano, on='Ano')

NameError: name 'resumo_cultura_total_ano' is not defined

### 2.2 Valor empenhado em Cultura em cada ano (2010-2022) em relação ao que estava previsto na Lei Orçamentária Anual (LOA) <a name="valor-empenhado-loa"></a>

In [41]:
cultura_por_ano = wsDf.loc[(wsDf['descrição da despesa'] == 'CULTURA') &(wsDf['mes']== 'dez')].sort_values(by='ano')
cultura_por_ano['pct']  = round(cultura_por_ano['Valor Acumulado']/cultura_por_ano['Valor Anual Previsto na LOA']*100,2)
cultura_por_ano=cultura_por_ano[['ano','Valor Anual Previsto na LOA','Valor Acumulado','pct']]
cultura_por_ano.columns = ['Ano','Valor previsto na LOA','Valor empenhado no ano','Pct']
cultura_por_ano.to_csv(arquivos + 'valor_anual_previsto_LOA_e_valor_acumulado.csv', index=False)
cultura_por_ano

,Ano,Valor previsto na LOA,Valor empenhado no ano,Pct
9,2010,63770546.00,31217869.22,48.95
9,2011,78348304.00,41761489.58,53.30
9,2012,99861612.00,50230328.42,50.30
9,2013,90266544.00,36721858.05,40.68
11,2014,57645418.00,44820159.52,77.75
10,2015,57985508.00,32255792.19,55.63
10,2016,88398641.00,46434616.84,52.53
10,2017,96365773.00,43347750.02,44.98
10,2018,95069137.00,60060690.19,63.18
10,2019,104948694.00,72309943.34,68.90


### 2.3 Valores Empenhados, Liquidados e Pagos para a Cultura, entre 2014 e 2022 <a name="valores-empenhados-liquidados-pagos"></a>
(Valores totais e distribuídos por subfunção)


In [42]:
valores_cultura_soma = subfuncoes_cultura.groupby('ano').sum().round(2)
valores_cultura_soma

,Dotação Orçamentária,Valor Empenhado,Valor Liquidado,Valor Pago
ano,,,,
2014,30293643.00,17959184.64,17959184.64,15888448.95
2015,24532065.00,21236437.43,21132017.44,20724053.97
2016,36537654.00,27374169.02,27008773.64,26971005.10
2017,29008519.00,27659035.17,27437682.64,27380270.86
2018,17328743.00,15876914.38,15729878.00,15689393.70
2019,43945423.00,40134986.66,38109576.55,37869534.93
2020,37927557.00,34162964.22,33041567.19,32994905.57
2021,23992607.00,18593628.52,16668781.43,16373598.47
2022,41801420.00,26512144.03,20563025.45,19885172.83


In [43]:
valores_cultura = df[df['função / subfunção'] == 'CULTURA'][['ano','Dotação Orçamentária', 'Valor Empenhado','Valor Liquidado',
       'Valor Pago','%(B/A)%','valor_pago_corrigido','valor_empenhado_corrigido','valor_liquidado_corrigido']].sort_values(by='ano')
valores_cultura

KeyError: "['valor_pago_corrigido', 'valor_empenhado_corrigido', 'valor_liquidado_corrigido'] not in index"

In [44]:
# Teste de valores totais para cultura e divididos por subfunção, a cada ano:

valores_cultura_soma.reset_index()['Valor Pago'] == valores_cultura.reset_index()['Valor Pago']

NameError: name 'valores_cultura' is not defined

In [45]:
valores_cultura_soma.reset_index()['Valor Liquidado'] == valores_cultura.reset_index()['Valor Liquidado']

NameError: name 'valores_cultura' is not defined

In [46]:
valores_cultura_soma.reset_index()['Valor Empenhado'] == valores_cultura.reset_index()['Valor Empenhado']

NameError: name 'valores_cultura' is not defined

In [47]:
valores_cultura_soma.reset_index()['Valor Pago'] == valores_cultura.reset_index()['Valor Pago']

NameError: name 'valores_cultura' is not defined

In [48]:
colunas = ['ano','Dotação Orçamentária', 'Valor Empenhado', 'Valor Liquidado', 'Valor Pago','função / subfunção','valor_pago_corrigido','valor_empenhado_corrigido','valor_liquidado_corrigido']

todas = df[df['função / subfunção'] == 'CULTURA'][colunas].copy().sort_values(by='ano')
todas['função / subfunção'] = 'Todas'

pd.concat([todas,subfuncoes_cultura[colunas]]).to_csv(arquivos + 'gastos_cultura_por_subfuncao.csv', index=False)


pd.concat([todas,subfuncoes_cultura[colunas]])

KeyError: "['valor_pago_corrigido', 'valor_empenhado_corrigido', 'valor_liquidado_corrigido'] not in index"

In [49]:
resumo_por_subfuncao =  subfuncoes_cultura[colunas].copy()

resumo_por_subfuncao.loc[(resumo_por_subfuncao['função / subfunção'] != 'ADMINISTRACAO GERAL') & (resumo_por_subfuncao['função / subfunção'] != 'DIFUSAO CULTURAL'),'função / subfunção'] = 'Outras'

resumo_por_subfuncao = resumo_por_subfuncao.groupby(['ano','função / subfunção']).sum().reset_index().pivot(index='ano', columns='função / subfunção', values='valor_pago_corrigido').fillna(0).reset_index()

KeyError: "['valor_pago_corrigido', 'valor_empenhado_corrigido', 'valor_liquidado_corrigido'] not in index"

In [50]:
resumo_por_subfuncao

NameError: name 'resumo_por_subfuncao' is not defined

In [51]:
resumo_por_subfuncao.to_csv(arquivos + 'resumo_por_subfuncao.csv', index=False)

NameError: name 'resumo_por_subfuncao' is not defined

In [52]:
resumo_por_subfuncao_v2 =  subfuncoes_cultura[colunas].copy()

resumo_por_subfuncao_v2 = resumo_por_subfuncao_v2.groupby(['ano','função / subfunção']).sum().reset_index().pivot(index='ano', columns='função / subfunção', values='valor_pago_corrigido').fillna(0).reset_index()

KeyError: "['valor_pago_corrigido', 'valor_empenhado_corrigido', 'valor_liquidado_corrigido'] not in index"

In [53]:
resumo_por_subfuncao_v2.to_csv(arquivos + 'resumo_por_subfuncao_v2.csv', index=False)

NameError: name 'resumo_por_subfuncao_v2' is not defined

In [54]:
resumo_por_subfuncao_v2

NameError: name 'resumo_por_subfuncao_v2' is not defined